<a href="https://colab.research.google.com/github/hsynj/AI-Special-HW/blob/main/Movie%20RAG%20Recommender/01_Movie_RAG_Recommender(Advanced).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced RAG Pipeline for Movie Recommendations

This notebook takes the `simple_rag_exercise.ipynb` as a starting point and enhances it with several advanced RAG techniques to improve accuracy, context-awareness, and interactivity.

This project demonstrates a more robust, production-ready approach to RAG.

### Key Enhancements:
1.  **Richer Embeddings:**
    * Instead of embedding *only* the `overview`, this pipeline combines `title`, `genres`, `keywords`, and `overview` into a single, rich document for embedding. This allows for much more complex and accurate queries (e.g., searching by genre *and* story).

2.  **Re-Ranking (Cross-Encoder):**
    * To improve precision, the pipeline first retrieves the Top-K (e.g., Top 5) candidates from Qdrant.
    * It then uses a `Cross-Encoder` (Re-Ranker) to score the relevance of each candidate against the query *directly*.
    * This finds the *single best match* and solves the "Top-1 limitation" of a simple vector search.
3.  **Conversational Memory:**
    * The pipeline now includes a `chat_history` list.
    * This history is fed back into the LLM prompt, allowing the system to remember previous interactions and handle follow-up questions (e.g., "Thanks, find me another one...").

4.  **Flexible Generation:**
    * The generation step is configured to use Google's `gemini-pro` API.

In [ ]:
# requirements
!pip install datasets qdrant-client sentence-transformers google-generativeai openai rich

In [ ]:
from rich.console import Console

# Create a console (we will use the default theme)
console = Console()

In [ ]:
from datasets import load_dataset

dataset = load_dataset('AiresPucrs/tmdb-5000-movies')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
console.print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'budget', 'genres', 'homepage', 'keywords', 'original_language', 'original_title', 
'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 
'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count', 'cast', 'crew'],
        num_rows: 4803
    })
})

In [ ]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

# Create the embedding encoder
encoder = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [ ]:
console.print(encoder)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False, 'architecture': 'MPNetModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True,
'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': 
False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
# Create collection to store the wine rating data
collection_name="movies"

qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/tmp/ipython-input-2094978642.py:4: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [ ]:
### [ADVANCED] Helper function to create a rich document for embedding.
import json

def create_movie_document(movie):
    try:
        genres_list = json.loads(movie['genres'])
    except (TypeError, json.JSONDecodeError):
        genres_list = movie['genres'] if isinstance(movie['genres'], list) else []

    try:
        keywords_list = json.loads(movie['keywords'])
    except (TypeError, json.JSONDecodeError):
        keywords_list = movie['keywords'] if isinstance(movie['keywords'], list) else []

    genres = ", ".join([g['name'] for g in genres_list if g and 'name' in g])
    keywords = ", ".join([k['name'] for k in keywords_list if k and 'name' in k])

    return f"Title: {movie['original_title']}. Genres: {genres}. Keywords: {keywords}. Overview: {movie['overview']}"

In [ ]:
points_list = []
for i, movie in enumerate(dataset['train']):
  if movie['overview']:
    print(f"{i+1} - Processing ID: {movie['id']}, Title: {movie['original_title']}")
    points_list.append(
        models.PointStruct(
            id=movie['id'],
            vector=encoder.encode(create_movie_document(movie)).tolist(),
            payload=movie
        )
    )

print("\nStarting upload to Qdrant...")
qdrant.upload_points(
    collection_name=collection_name,
    points=points_list
)
print("Upload complete!")

1 - Processing ID: 5, Title: Four Rooms
2 - Processing ID: 11, Title: Star Wars
3 - Processing ID: 12, Title: Finding Nemo
4 - Processing ID: 13, Title: Forrest Gump
5 - Processing ID: 14, Title: American Beauty
6 - Processing ID: 16, Title: Dancer in the Dark
7 - Processing ID: 18, Title: The Fifth Element
8 - Processing ID: 19, Title: Metropolis
9 - Processing ID: 20, Title: My Life Without Me
10 - Processing ID: 22, Title: Pirates of the Caribbean: The Curse of the Black Pearl
11 - Processing ID: 24, Title: Kill Bill: Vol. 1
12 - Processing ID: 25, Title: Jarhead
13 - Processing ID: 28, Title: Apocalypse Now
14 - Processing ID: 33, Title: Unforgiven
15 - Processing ID: 35, Title: The Simpsons Movie
16 - Processing ID: 38, Title: Eternal Sunshine of the Spotless Mind
17 - Processing ID: 55, Title: Amores perros
18 - Processing ID: 58, Title: Pirates of the Caribbean: Dead Man's Chest
19 - Processing ID: 59, Title: A History of Violence
20 - Processing ID: 62, Title: 2001: A Space Ody

In [ ]:
console.print(
    qdrant
    .get_collection(
        collection_name=collection_name
    )
)

CollectionInfo(
    status=<CollectionStatus.GREEN: 'green'>,
    optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>,
    vectors_count=None,
    indexed_vectors_count=0,
    points_count=4800,
    segments_count=1,
    config=CollectionConfig(
        params=CollectionParams(
            vectors=VectorParams(
                size=768,
                distance=<Distance.COSINE: 'Cosine'>,
                hnsw_config=None,
                quantization_config=None,
                on_disk=None,
                datatype=None,
                multivector_config=None
            ),
            shard_number=None,
            sharding_method=None,
            replication_factor=None,
            write_consistency_factor=None,
            read_fan_out_factor=None,
            on_disk_payload=None,
            sparse_vectors=None
        ),
        hnsw_config=HnswConfig(
            m=16,
            ef_construct=100,
            full_scan_threshold=10000,
            max_indexing_threads=0,
            on_disk=None,
            payload_m=None
        ),
        optimizer_config=OptimizersConfig(
            deleted_threshold=0.2,
            vacuum_min_vector_number=1000,
            default_segment_number=0,
            max_segment_size=None,
            memmap_threshold=None,
            indexing_threshold=20000,
            flush_interval_sec=5,
            max_optimization_threads=1
        ),
        wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0),
        quantization_config=None,
        strict_mode_config=None
    ),
    payload_schema={}
)

In [ ]:
user_prompt = "Thanks! Now find a comedy from the 2000s"

In [ ]:
query_vector = encoder.encode(user_prompt).tolist()

In [ ]:
from qdrant_client import models

query_filter= models.Filter(
  must=[
      models.FieldCondition(
          key='release_date',
          range=models.DatetimeRange(
              gte='1990-01-01T00:00:00Z',
              lte='1999-12-31T23:59:59Z'
          )
      )
  ]
)

In [ ]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=5,
    query_filter=query_filter,
)

/tmp/ipython-input-1189791493.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


In [ ]:
from rich.text import Text
from rich.table import Table

table = Table(title="Retrieval Results", show_lines=True)

table.add_column("ID", style="#e0e0e0")
table.add_column("Original Title", style="#e0e0e0")
table.add_column("Overview", style="bright_red")
table.add_column("Score", style="#89ddff")

for hit in hits:
    table.add_row(
        str(hit.payload["id"]),
        hit.payload["original_title"],
        f'{hit.payload["overview"]}',
        f"{hit.score:.4f}"
    )

console.print(table)

                                                 Retrieval Results                                                 
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ ID    ┃ Original Title   ┃ Overview                                                                    ┃ Score  ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 11676 │ Nothing to Lose  │ Advertising executive Nick Beame learns that his wife is sleeping with his  │ 0.4342 │
│       │                  │ employer. In a state of despair, he encounters a bumbling thief whose       │        │
│       │                  │ attempted carjacking goes awry when Nick takes him on an involuntary        │        │
│       │                  │ joyride. Soon the betrayed businessman and the incompetent crook strike up  │        │
│       │                  │ a partnership and develop a robbery-revenge scheme. But it turns out that   │        │
│       │                  │ some other criminals in the area don't appreciate the competition.          │        │
├───────┼──────────────────┼─────────────────────────────────────────────────────────────────────────────┼────────┤
│ 14022 │ Slacker          │ Presents a day in the life in Austin, Texas among its social outcasts and   │ 0.4321 │
│       │                  │ misfits, predominantly the twenty-something set, using a series of linear   │        │
│       │                  │ vignettes. These characters, who in some manner just don't fit into the     │        │
│       │                  │ establishment norms, move seamlessly from one scene to the next, randomly   │        │
│       │                  │ coming and going into one another's lives.                                  │        │
├───────┼──────────────────┼─────────────────────────────────────────────────────────────────────────────┼────────┤
│ 2293  │ Mallrats         │ Both dumped by their girlfriends, two best friends seek refuge in the local │ 0.4277 │
│       │                  │ mall.                                                                       │        │
├───────┼──────────────────┼─────────────────────────────────────────────────────────────────────────────┼────────┤
│ 8467  │ Dumb and Dumber  │ Lloyd and Harry are two men whose stupidity is really indescribable. When   │ 0.4199 │
│       │                  │ Mary, a beautiful woman, loses an important suitcase with money before she  │        │
│       │                  │ leaves for Aspen, the two friends (who have found the suitcase) decide to   │        │
│       │                  │ return it to her. After some "adventures" they finally get to Aspen where,  │        │
│       │                  │ using the lost money they live it up and fight for Mary's heart.            │        │
├───────┼──────────────────┼─────────────────────────────────────────────────────────────────────────────┼────────┤
│ 2277  │ Bicentennial Man │ Richard Martin buys a gift, a new NDR-114 robot. The product is named       │ 0.4125 │
│       │                  │ Andrew by the youngest of the family's children. "Bicentennial Man" follows │        │
│       │                  │ the life and times of Andrew, a robot purchased as a household appliance    │        │
│       │                  │ programmed to perform menial tasks. As Andrew begins to experience emotions │        │
│       │                  │ and creative thought, the Martin family soon discovers they don't have an   │        │
│       │                  │ ordinary robot.                                                             │        │
└───────┴──────────────────┴─────────────────────────────────────────────────────────────────────────────┴────────┘

In [ ]:
### [ADVANCED] Re-rank the top-K retrieval results for higher precision.

from sentence_transformers.cross_encoder import CrossEncoder

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

query = user_prompt
pairs = [(query, hit.payload['overview']) for hit in hits]

scores = cross_encoder.predict(pairs)

best_hit = hits[scores.argmax()]

In [ ]:
### [ADVANCED] Display the single best re-ranked result.
from rich.table import Table
from rich.console import Console

if 'best_hit' in locals() and best_hit is not None:

    table = Table(
        title="Best Hit Details",
        show_header=True,
        header_style="bold magenta",
        show_lines=True,
        padding=(0, 1)
    )

    table.add_column("Field", style="dim", width=18)
    table.add_column("Value", style="cyan")
    table.add_row("Qdrant ID", str(best_hit.id))
    table.add_row("Relevance Score", f"{best_hit.score:.4f}")
    payload = best_hit.payload

    if 'original_title' in payload:
        table.add_row("Original Title", payload['original_title'])

    if 'release_date' in payload:
        table.add_row("Release Date", payload['release_date'])

    if 'popularity' in payload:
        table.add_row("Popularity", f"{payload['popularity']:.2f}")

    if 'genres' in payload:
        table.add_row("Genres", str(payload['genres']))

    if 'overview' in payload:
        table.add_row("Overview", payload['overview'])

    console.print(table)

else:
    console.print("[bold red]Error:[/bold red] The variable 'best_hit' was not found or is empty.")

                                                 Best Hit Details                                                  
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Field              ┃ Value                                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Qdrant ID          │ 14022                                                                                      │
├────────────────────┼────────────────────────────────────────────────────────────────────────────────────────────┤
│ Relevance Score    │ 0.4321                                                                                     │
├────────────────────┼────────────────────────────────────────────────────────────────────────────────────────────┤
│ Original Title     │ Slacker                                                                                    │
├────────────────────┼────────────────────────────────────────────────────────────────────────────────────────────┤
│ Release Date       │ 1990-07-27                                                                                 │
├────────────────────┼────────────────────────────────────────────────────────────────────────────────────────────┤
│ Popularity         │ 3.32                                                                                       │
├────────────────────┼────────────────────────────────────────────────────────────────────────────────────────────┤
│ Genres             │ [{"id": 35, "name": "Comedy"}]                                                             │
├────────────────────┼────────────────────────────────────────────────────────────────────────────────────────────┤
│ Overview           │ Presents a day in the life in Austin, Texas among its social outcasts and misfits,         │
│                    │ predominantly the twenty-something set, using a series of linear vignettes. These          │
│                    │ characters, who in some manner just don't fit into the establishment norms, move           │
│                    │ seamlessly from one scene to the next, randomly coming and going into one another's lives. │
└────────────────────┴────────────────────────────────────────────────────────────────────────────────────────────┘

In [ ]:
# define a variable to hold the search results with specific fields
search_results = [
    {
        'original_title': hit.payload['original_title'],
        'title': hit.payload['title'],
        'overview': hit.payload['overview'],
        'release_date': hit.payload['release_date'],
        'popularity': hit.payload['popularity']
    } for hit in hits]

In [ ]:
console.print(search_results)

[
    {
        'original_title': 'Nothing to Lose',
        'title': 'Nothing to Lose',
        'overview': "Advertising executive Nick Beame learns that his wife is sleeping with his employer. In a 
state of despair, he encounters a bumbling thief whose attempted carjacking goes awry when Nick takes him on an 
involuntary joyride. Soon the betrayed businessman and the incompetent crook strike up a partnership and develop a 
robbery-revenge scheme. But it turns out that some other criminals in the area don't appreciate the competition.",
        'release_date': '1997-07-18',
        'popularity': 8.618208
    },
    {
        'original_title': 'Slacker',
        'title': 'Slacker',
        'overview': "Presents a day in the life in Austin, Texas among its social outcasts and misfits, 
predominantly the twenty-something set, using a series of linear vignettes. These characters, who in some manner 
just don't fit into the establishment norms, move seamlessly from one scene to the next, randomly coming and going 
into one another's lives.",
        'release_date': '1990-07-27',
        'popularity': 3.320622
    },
    {
        'original_title': 'Mallrats',
        'title': 'Mallrats',
        'overview': 'Both dumped by their girlfriends, two best friends seek refuge in the local mall.',
        'release_date': '1995-10-20',
        'popularity': 16.139771
    },
    {
        'original_title': 'Dumb and Dumber',
        'title': 'Dumb and Dumber',
        'overview': 'Lloyd and Harry are two men whose stupidity is really indescribable. When Mary, a beautiful 
woman, loses an important suitcase with money before she leaves for Aspen, the two friends (who have found the 
suitcase) decide to return it to her. After some "adventures" they finally get to Aspen where, using the lost money
they live it up and fight for Mary\'s heart.',
        'release_date': '1994-12-16',
        'popularity': 35.777067
    },
    {
        'original_title': 'Bicentennial Man',
        'title': 'Bicentennial Man',
        'overview': 'Richard Martin buys a gift, a new NDR-114 robot. The product is named Andrew by the youngest 
of the family\'s children. "Bicentennial Man" follows the life and times of Andrew, a robot purchased as a 
household appliance programmed to perform menial tasks. As Andrew begins to experience emotions and creative 
thought, the Martin family soon discovers they don\'t have an ordinary robot.',
        'release_date': '1999-12-17',
        'popularity': 24.939295
    }
]

In [ ]:
### [ADVANCED] Implement conversational memory (chat history) for follow-ups.
if 'chat_history' not in globals():
    chat_history = []

In [ ]:
import google.generativeai as genai
from google.colab import userdata
from rich.panel import Panel
from rich.text import Text

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
except Exception as e:
    console.print(f"[bold red]Error:[/bold red] Could not configure Gemini. Make sure you have stored 'GOOGLE_API_KEY' in Colab Secrets.")
    raise e

history_string = "\n".join([f"User: {turn['user']}\nModel: {turn['model']}" for turn in chat_history])
system_prompt = "You are a helpful movie recommendation assistant..."
user_request = f"The user is asking for: '{user_prompt}'."
context = f"Based on this, I found the following movie context: {search_results}."
full_prompt = f"Previous conversation:\n{history_string}\n\n{system_prompt}\n\n{context}\n\n{user_request}\n\nProvide a recommendation for the user based *only* on the new context and be aware of the previous conversation."


model = genai.GenerativeModel('gemini-2.5-flash')
completion = model.generate_content(full_prompt)

response_text = Text(completion.text)
styled_panel = Panel(
    response_text,
    title="Movie Recommendation with Retrieval",
    expand=False,
    border_style="bright_yellow",
    padding=(1, 1)
)

response_text = completion.text
chat_history.append({"user": user_prompt, "model": response_text})

console.print(styled_panel)

╭────────────────────────────────────── Movie Recommendation with Retrieval ──────────────────────────────────────╮
│                                                                                                                 │
│ I apologize, but none of the movies in the provided context are comedies released in the 2000s. All the         │
│ comedies listed are from the 1990s.                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
### [ADVANCED] Display the full conversational chat history log.
from rich.console import Console
from rich.table import Table

if 'chat_history' not in locals() or not chat_history:
    console.print("[yellow]Alert: 'chat_history' not found. Using sample data for demonstration.[/yellow]\n")
    chat_history = [
        {"user": "Love story between an Asian king and European teacher",
         "model": "Based on your request, I recommend 'Anna and the King' (1999). It's a romance about the King of Siam and a British school teacher..."},
        {"user": "Thanks! Can you find me another one like that, but a comedy?",
         "model": "Based on the new context, I recommend 'Crazy Rich Asians'. It's a modern romantic comedy..."}
    ]

if 'chat_history' in locals() and chat_history:
    table = Table(
        title="💬 Chat History Log",
        show_header=True,
        header_style="bold magenta",
        show_lines=True
    )
    table.add_column("Role", style="dim", width=15)
    table.add_column("Message", style="cyan")

    for i, turn in enumerate(chat_history):
        table.add_row(f"User (Turn {i+1})", turn['user'])
        table.add_row(f"Model (Turn {i+1})", turn['model'], end_section=True)
    console.print(table)

else:
    console.print("[bold red]Chat history is empty.[/bold red]")

                                                💬 Chat History Log                                                
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Role            ┃ Message                                                                                       ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ User (Turn 1)   │ Love story between an Asian king and European teacher                                         │
├─────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ Model (Turn 1)  │ Based on your request, I recommend:                                                           │
│                 │                                                                                               │
│                 │ **Anna and the King**                                                                         │
│                 │                                                                                               │
│                 │ This film tells the story of the romance between the King of Siam (now Thailand) and the      │
│                 │ widowed British school teacher Anna Leonowens during the 1860s. Anna teaches the children and │
│                 │ becomes romanced by the King.                                                                 │
├─────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ User (Turn 2)   │ Thanks! Now find a comedy from the 2000s                                                      │
├─────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ Model (Turn 2)  │ I apologize, but none of the movies in the provided context are comedies released in the      │
│                 │ 2000s. All the comedies listed are from the 1990s.                                            │
└─────────────────┴───────────────────────────────────────────────────────────────────────────────────────────────┘